# Load all the links into Datasette for exploration

This notebook brings together the harvested datasets, does a bit of cleaning up and reorganisation, then loads everything into an SQLite database that can be opened using Datasette.

If you update the harvested datasets, then you'll need to change the file names in the code.

In [ ]:
import pandas as pd
from sqlite_utils import Database
from pathlib import Path

In [ ]:
Path("datasette/acde-links.db").unlink(missing_ok=True)
db = Database("datasette/acde-links.db")

In [ ]:
# Prepare DAAO WD (add in DAAO ids and reorder)
daao_wd = pd.read_json("daao_wd_all_links_20221006.json")
# Replace the DAAO path segments with hash identifiers
# This is so we can link records back to the ACDE dataset which uses the hashes
daao_wd.rename(columns={"or_id": "or_path"}, inplace=True)
daao_urls = pd.read_csv("daao_urls.csv")
daao_merged = pd.merge(
    daao_wd, daao_urls, how="left", left_on="or_path", right_on="daao_path"
)
daao_merged.rename(columns={"daao_id": "or_id", "daao_url": "or_url"}, inplace=True)

# Clean up and reorder
daao_merged = daao_merged[
    [
        "acde_source",
        "or_id",
        "or_url",
        "wd_url",
        "wd_label",
        "related_source",
        "related_id",
        "related_url",
    ]
]
daao_merged.to_json("daao_wd_all_links_datasette.json", orient="records")

In [ ]:
# Prepare AS wikidata (reorder columns)
ausstage_wd = pd.read_json("ausstage_wd_all_links_20221006.json")
ausstage_wd["or_url"] = ausstage_wd["or_id"].apply(
    lambda x: f"https://www.ausstage.edu.au/pages/contributor/{x}")
ausstage_wd = ausstage_wd[
    [
        "acde_source",
        "or_id",
        "or_url",
        "wd_url",
        "wd_label",
        "related_source",
        "related_id",
        "related_url",
    ]
]
ausstage_wd.to_json("ausstage_wd_all_links_datasette.json", orient="records")

In [ ]:
acde_df = pd.read_json("ACDE_Merged_Normalized_202206031344.json")
# Add in the DAAO urls
daao_urls.drop_duplicates(subset="daao_id", inplace=True)
acde_merged = pd.merge(
    acde_df, daao_urls, how="left", left_on="ori_id", right_on="daao_id"
)
acde_merged.rename(columns={"daao_url": "ori_url"}, inplace=True)
acde_merged = acde_merged[
    [
        "data_source",
        "ori_id",
        "ori_url",
        "display_name",
        "first_name",
        "middle_name",
        "last_name",
        "other_names",
        "roles",
        "gender",
        "nationality",
        "year_of_birth",
        "year_of_death",
    ]
]

In [ ]:
# Prepare ACDE
acde_df = pd.read_json("ACDE_Merged_Normalized_202206031344.json")
# Add in the DAAO urls
daao_urls.drop_duplicates(subset="daao_id", inplace=True)
acde_merged = pd.merge(
    acde_df, daao_urls, how="left", left_on="ori_id", right_on="daao_id"
)
acde_merged.rename(columns={"daao_url": "ori_url"}, inplace=True)
acde_datasette = acde_merged[
    [
        "data_source",
        "ori_id",
        "ori_url",
        "display_name",
        "first_name",
        "middle_name",
        "last_name",
        "other_names",
        "roles",
        "gender",
        "nationality",
        "year_of_birth",
        "year_of_death",
    ]
]
# Add AusStage urls
acde_datasette.loc[acde_datasette["data_source"] == "AusStage", "ori_url"] = acde_datasette["ori_id"].apply(
    lambda x: f"https://www.ausstage.edu.au/pages/contributor/{x}"
)
acde_datasette.loc[acde_datasette["data_source"] == "DAQA", "ori_url"] = acde_datasette["ori_id"].apply(
    lambda x: f"https://qldarch.net/architect/summary?architectId={x}"
)
acde_datasette.to_json(
    "ACDE_Merged_Normalized_datasette.json", orient="records"
)

In [ ]:
def add_key(file, id_col="or_id", col_name="acde_id", cols=["acde_source", "or_id"], drop=["or_id"]):
    df = pd.read_json(file)
    keys = df[cols].apply(
        lambda x: "-".join(x.astype("str")) if x[id_col] else "", axis=1
    )
    df.insert(loc=2, column=col_name, value=keys)
    df.drop(columns=drop, inplace=True)
    return df.to_dict("records")

In [ ]:
# Add keys based on ACDE source and ori_id that can be used to link tables
acde_dataset = add_key(
    "ACDE_Merged_Normalized_datasette.json",
    id_col="ori_id",
    col_name="key",
    cols=["data_source", "ori_id"],
    drop=[]
)
ausstage_trove_links = add_key("ausstage_trove_links_20221005.json")
ausstage_trove_relations = add_key("ausstage_trove_relations_20221005.json")
ausstage_trove_resources = add_key("ausstage_trove_resources_20221005.json")
ausstage_wd_links = add_key("ausstage_wd_all_links_datasette.json")
daao_trove_links = add_key("daao_trove_links_20221004.json")
daao_trove_relations = add_key("daao_trove_relations_20221004.json")
daao_trove_resources = add_key("daao_trove_resources_20221004.json")
daao_wd_links = add_key("daao_wd_all_links_datasette.json")
daqa_trove_matches = add_key("daqa_trove_matches_20221006.json")
daao_viaf_links = add_key("daao_viaf_links_20221006.json")
ausstage_viaf_links = add_key("ausstage_viaf_links_20221006.json")

In [ ]:
db["acde_records"].insert_all(acde_dataset, pk="key")

In [ ]:
db["trove_links"].insert_all(
    ausstage_trove_links, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["trove_resources"].insert_all(
    ausstage_trove_resources, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["trove_relations"].insert_all(
    ausstage_trove_relations, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["wikidata_links"].insert_all(
    ausstage_wd_links, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["viaf_links"].insert_all(
    ausstage_viaf_links, foreign_keys=[("acde_id", "acde_records", "key")]
)

In [ ]:
db["trove_links"].insert_all(
    daao_trove_links, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["trove_resources"].insert_all(
    daao_trove_resources, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["trove_relations"].insert_all(
    daao_trove_relations, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["wikidata_links"].insert_all(
    daao_wd_links, foreign_keys=[("acde_id", "acde_records", "key")]
)
db["viaf_links"].insert_all(
    daao_viaf_links, foreign_keys=[("acde_id", "acde_records", "key")]
)

In [ ]:
db["daqa_matches"].insert_all(
    daqa_trove_matches, foreign_keys=[("acde_id", "acde_records", "key")]
)